In [2]:
import json
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

In [14]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "company": {"type": "text"},
            "name": {"type": "text"},
            "specifications": {"type": "keyword"} ,
            "name_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
            "specification_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"}
        }
    }
}

In [5]:
es_client = Elasticsearch('http://localhost:9200')

In [15]:
index_name = "mobile-specifications"
es_client.indices.delete(index=index_name)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'mobile-specifications'})

In [7]:
import pickle

In [10]:
with open('mobile_specs_vector_data.pkl', 'rb') as fp:
    mobile_specs_vector_data = pickle.load(fp)

In [11]:
bkp_mobile_specs_vector_data[0].keys()

dict_keys(['company', 'name', 'specifications', 'name_vector', 'specification_vector'])

In [12]:
mobile_specs_vector_data[0].keys()

dict_keys(['company', 'name', 'specifications', 'name_vector', 'specification_vector'])

In [16]:
for doc in tqdm(mobile_specs_vector_data):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2096/2096 [00:30<00:00, 67.79it/s]


In [21]:
from huggingface_hub import InferenceClient

In [22]:
import os

In [23]:
HF_API_TOKEN = os.getenv('HF_API_TOKEN')
client = InferenceClient("HuggingFaceH4/zephyr-7b-beta", token=HF_API_TOKEN)

In [27]:
model = SentenceTransformer("all-mpnet-base-v2")

/Users/amoghkulkarni/.local/share/virtualenvs/gadget-spec-bot-PKdu65VV/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [24]:
def elastic_search_knn(vector, vector_field="name_vector"):
    """Return elastic search results for given vector."""
    knn_query = {
        "field": vector_field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000
    }

    search_query = {
        'knn': knn_query,
        '_source': ["name", "company", "specifications"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )

    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [25]:
def get_mobile_name_results(mobile_name_statement):
    """Return the ES results of the mobile phone asked by user."""
    input_vector = model.encode(mobile_name_statement)
    mobile_context = elastic_search_knn(input_vector, 'specification_vector')

    return mobile_context

In [18]:
def build_prompt(phone_specifications, mobile_context):
    """Build the context using search results and return the prompt."""
    prompt_template = """
You are mobile phone expert. Answer all the specifications of PHONE using the CONTEXT provided from SPECFICATIONS database.
The specifications in the CONTEXT is a json string. Convert the json string to readable format before printing out the answer.
Use only the specifications given in the context to print the specs of the device.
Do not number the specifications passed.
Print only the first result in a markdown format where subsections are indented in readable format.


PHONE: {phone}

CONTEXT:
{context}
""".strip()

    context = ""

    for each in mobile_context:
        context = context + f"\n\nname: {each['name']}\ncompany: {each['company']}\nspecifications: {each['specifications']}"

    prompt = prompt_template.format(phone=phone_specifications, context=context)
    return prompt

In [19]:
def llm(prompt):
    """Return LLM generation based on context."""
    messages = [{"role": "user", "content": prompt}]
    response = client.chat_completion(messages, max_tokens=1000)
    
    return response.choices[0].message.content

In [20]:
def rag(mobile_phone_statement):
    """Run the RAG flow for the given mobile phone statement."""
    mobile_name_context = get_mobile_name_results(mobile_phone_statement)
    prompt = build_prompt(mobile_phone_statement, mobile_name_context)
    answer = llm(prompt)

    return answer

In [28]:
print(rag('Tell me the specifications of iphone 15'))

IPhone 15:

- Brand: Apple
- Model: iPhone 15
- Price in India: ₹69,900
- Release date: 12th September 2023
- Launched in India: Yes
- Dimensions (mm): 147.60 x 71.60 x 7.80
- Weight (g): 171.00
- IP rating: IP68
- Removable battery: No
- Wireless charging: Yes
- Colors: Black, Blue, Green, Pink, Yellow

Display:
- Refresh Rate: 60 Hz
- Screen size (inches): 6.10
- Touchscreen: Yes
- Resolution: 1179x2556 pixels
- Pixels per inch (PPI): 460

Hardware:
- Processor: hexa-core
- Processor make: Apple A16 Bionic
- RAM: 6GB
- Internal storage: 128GB, 256GB, 512GB
- Expandable storage: No

Camera:
- Rear camera: 48-megapixel (f/1.6) + 12-megapixel (f/2.4)
- No. Of Rear Cameras: 2
- Front camera: 12-megapixel (f/1.9)
- No. Of Front Cameras: 1

Software:
- Operating system: iOS 17

Connectivity:
- Wi-Fi: Yes
- Wi-Fi standards supported: 802.11 b/g/n/ac/ax
- GPS: Yes
- Bluetooth: Yes, v 5.30
- NFC: Yes
- Infrared: No
- USB Type-C: No
- Headphones: Lightning
- Number of SIMs: 2
- Active 4G on bo

In [30]:
print(rag('Which is the best phone with 128-megapixel camera?'))

PHONE with 128-megapixel camera: The Nokia X100 is the best phone with a 128-megapixel camera, as specified in the context provided. It has a quad camera set-up at the rear with a primary sensor of 128-megapixels, alongside 5-megapixel, 2-megapixel and 2-megapixel lenses. This stunning phone from Nokia, released on November 10, 2021, comes with a 6.67-inch FHD+ display, a Qualcomm Snapdragon 480 processor, and 6GB of RAM. The Nokia X100 runs on Android 11 and has a large 4470mAh battery with charging and wireless charging capabilities. Its design includes a sleek 9.10mm thickness and comes in different hues such as Midnight Blue. Make the Nokia X100 your go-to option for an exemplary camera experience.


## Going ahead with specification vector search as it gives me "almost" right answers with different prompts